In [17]:
import os.path
import numpy as np
import pandas as pd

In [18]:
BPL1314 = pd.read_html("https://www.statbunker.com/competitions/FantasyFootballPlayersStats?comp_id=449")[0]
BUN1314 = pd.read_html("https://www.statbunker.com/competitions/FantasyFootballPlayersStats?comp_id=447")[0]
LAL1314 = pd.read_html("https://www.statbunker.com/competitions/FantasyFootballPlayersStats?comp_id=461")[0]
SEI1314 = pd.read_html("https://www.statbunker.com/competitions/FantasyFootballPlayersStats?comp_id=462")[0]
FRA1314 = pd.read_html("https://www.statbunker.com/competitions/FantasyFootballPlayersStats?comp_id=454")[0]

In [19]:
SEA1314 = [BPL1314,BUN1314,LAL1314,SEI1314,FRA1314]
SEA1314 = pd.concat(SEA1314)
SEA1314 = SEA1314.reset_index()

In [20]:
SEA1314.head()

,index,Players,Points,Clubs,Position,Start,Goals,A,CS,CS part,...,Red,Sub,CO,Off,Pen SV,Pen M,Goals conceded,Conceded 1+,OG,More
0,0,Luis Suarez,254,LPOOL,Forward,33,31,13,-,-,...,0,0,0,3,-,0,-,-,0,More
1,1,Yaya Toure,193,MCFC,Midfielder,35,20,9,-,-,...,0,1,0,10,-,0,-,-,0,More
2,2,Daniel Sturridge,177,LPOOL,Forward,26,21,7,-,-,...,0,3,3,13,-,1,-,-,0,More
3,3,Olivier Giroud,172,ARSL,Forward,36,16,8,-,-,...,0,1,0,16,-,0,-,-,0,More
4,4,Steven Gerrard,162,LPOOL,Midfielder,33,13,13,-,-,...,0,1,1,6,-,1,-,-,0,More


In [21]:
SEA1314.drop(SEA1314.columns[[0,20]], axis=1, inplace=True)

In [22]:
SEA1314.head()

,Players,Points,Clubs,Position,Start,Goals,A,CS,CS part,Yellow,Red,Sub,CO,Off,Pen SV,Pen M,Goals conceded,Conceded 1+,OG
0,Luis Suarez,254,LPOOL,Forward,33,31,13,-,-,6,0,0,0,3,-,0,-,-,0
1,Yaya Toure,193,MCFC,Midfielder,35,20,9,-,-,4,0,1,0,10,-,0,-,-,0
2,Daniel Sturridge,177,LPOOL,Forward,26,21,7,-,-,2,0,3,3,13,-,1,-,-,0
3,Olivier Giroud,172,ARSL,Forward,36,16,8,-,-,4,0,1,0,16,-,0,-,-,0
4,Steven Gerrard,162,LPOOL,Midfielder,33,13,13,-,-,7,0,1,1,6,-,1,-,-,0


In [23]:
SEA1314['Apps'] = SEA1314['Start'] + SEA1314['CO']

In [24]:
SEA1314['Form'] = SEA1314['Points']/SEA1314['Apps']

In [25]:
SEA1314.loc[SEA1314['Form'].isnull(),'Form'] = 0

In [26]:
SEA1314.isnull().sum()

Players             0
Points              0
Clubs             579
Position            0
Start               0
Goals               0
A                   0
CS                  0
CS part             0
Yellow              0
Red                 0
Sub                 0
CO                  0
Off                 0
Pen SV              0
Pen M               0
Goals conceded      0
Conceded 1+         0
OG                  0
Apps                0
Form                0
dtype: int64

In [27]:
DashColumns = ['Goals','A','CS','CS part','Yellow','Red','Sub','CO','Off','Pen SV','Pen M','Goals conceded','Conceded 1+','OG',]
SEA1314[DashColumns] = SEA1314[DashColumns].replace('-' ,'0')

In [28]:
SEA1314.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2999 entries, 0 to 2998
Data columns (total 21 columns):
Players           2999 non-null object
Points            2999 non-null int64
Clubs             2420 non-null object
Position          2999 non-null object
Start             2999 non-null int64
Goals             2999 non-null int64
A                 2999 non-null int64
CS                2999 non-null object
CS part           2999 non-null object
Yellow            2999 non-null int64
Red               2999 non-null int64
Sub               2999 non-null int64
CO                2999 non-null int64
Off               2999 non-null int64
Pen SV            2999 non-null object
Pen M             2999 non-null int64
Goals conceded    2999 non-null object
Conceded 1+       2999 non-null object
OG                2999 non-null int64
Apps              2999 non-null int64
Form              2999 non-null float64
dtypes: float64(1), int64(12), object(8)
memory usage: 492.1+ KB


In [29]:
SEA1314['CS'] = SEA1314['CS'].str.replace('%','').astype(np.int)
SEA1314['CS part'] = SEA1314['CS part'].str.replace('%','').astype(np.int)
SEA1314['Pen SV'] = SEA1314['Pen SV'].str.replace('%','').astype(np.int)
SEA1314['Goals conceded'] = SEA1314['Goals conceded'].str.replace('%','').astype(np.int)
SEA1314['Conceded 1+'] = SEA1314['Conceded 1+'].str.replace('%','').astype(np.int)

In [30]:
SEA1314.loc[SEA1314['Players'].str.split().str.len() == 1, 'FirstInitial'] = SEA1314['Players'].astype(str).str[0]
SEA1314.loc[SEA1314['Players'].str.split().str.len() == 1, 'Firstname'] = SEA1314['Players'].str.split().str[0]
SEA1314.loc[SEA1314['Players'].str.split().str.len() == 1, 'Surname'] = SEA1314['Players'].str.split().str[0]

SEA1314.loc[SEA1314['Players'].str.split().str.len() == 2, 'FirstInitial'] = SEA1314['Players'].astype(str).str[0]
SEA1314.loc[SEA1314['Players'].str.split().str.len() == 2, 'Firstname'] = SEA1314['Players'].str.split().str[0]
SEA1314.loc[SEA1314['Players'].str.split().str.len() == 2, 'Surname'] = SEA1314['Players'].str.split().str[-1]

SEA1314.loc[SEA1314['Players'].str.split().str.len() == 3, 'FirstInitial'] = SEA1314['Players'].astype(str).str[0]
SEA1314.loc[SEA1314['Players'].str.split().str.len() == 3, 'Firstname'] = SEA1314['Players'].str.split().str[0]
SEA1314.loc[SEA1314['Players'].str.split().str.len() == 3, 'Surname'] = SEA1314['Players'].str.split().str[-1]

SEA1314.loc[SEA1314['Players'].str.split().str.len() == 4, 'FirstInitial'] = SEA1314['Players'].astype(str).str[0]
SEA1314.loc[SEA1314['Players'].str.split().str.len() == 4, 'Firstname'] = SEA1314['Players'].str.split().str[0]
SEA1314.loc[SEA1314['Players'].str.split().str.len() == 4, 'Surname'] = SEA1314['Players'].str.split().str[-1]

In [31]:
SEA1314.set_index(['Surname','FirstInitial'])

,,Players,Points,Clubs,Position,Start,Goals,A,CS,CS part,Yellow,...,CO,Off,Pen SV,Pen M,Goals conceded,Conceded 1+,OG,Apps,Form,Firstname
Surname,FirstInitial,,,,,,,,,,,,,,,,,,,,,
Suarez,L,Luis Suarez,254,LPOOL,Forward,33,31,13,0,0,6,...,0,3,0,0,0,0,0,33,7.696970,Luis
Toure,Y,Yaya Toure,193,MCFC,Midfielder,35,20,9,0,0,4,...,0,10,0,0,0,0,0,35,5.514286,Yaya
Sturridge,D,Daniel Sturridge,177,LPOOL,Forward,26,21,7,0,0,2,...,3,13,0,1,0,0,0,29,6.103448,Daniel
Giroud,O,Olivier Giroud,172,ARSL,Forward,36,16,8,0,0,4,...,0,16,0,0,0,0,0,36,4.777778,Olivier
Gerrard,S,Steven Gerrard,162,LPOOL,Midfielder,33,13,13,0,0,7,...,1,6,0,1,0,0,0,34,4.764706,Steven
Rooney,W,Wayne Rooney,160,MUFC,Forward,27,17,9,0,0,8,...,2,4,0,0,0,0,0,29,5.517241,Wayne
Lambert,R,Ricky Lambert,158,SOTOM,Forward,31,13,9,0,0,2,...,6,11,0,0,0,0,0,37,4.270270,Ricky
Hazard,E,Eden Hazard,156,CHEL,Midfielder,32,14,7,0,0,2,...,3,11,0,0,0,0,0,35,4.457143,Eden
Lukaku,R,Romelu Lukaku,154,CHEL,Forward,29,15,6,0,0,1,...,4,9,0,0,0,0,0,33,4.666667,Romelu


In [32]:
SEA1314.to_csv('../../data/prep/Fantasy1314.csv')